In [91]:
!pip install bs4
!pip install requests

In [53]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [54]:
url_to_main_page = "https://genshin-info.ru"
genshin_url = f"{url_to_main_page}/wiki/personazhi/"
html = requests.get(genshin_url).text
soup = BeautifulSoup(html, 'html')

In [55]:
itemcards = soup.find_all(lambda tag: tag.name == 'a' and tag.get('class') == ['itemcard'])
print("Всего персонажей:", len(itemcards))
print(itemcards[0])

Всего персонажей: 85
<a class="itemcard" href="/wiki/personazhi/arlekino/" id="bx_3218110189__1676533a4586dcea66c3b61d68a9762b" itemscope="" itemtype="http://schema.org/ImageObject" title="Арлекино">
<div class="itemcard__element _pyro"></div>
<div class="itemcard__sticker">Скоро</div>
<div class="itemcard__imgC _s5 _desc">
<div class="itemcard__img">
<img class="lazyload" data-src="/upload/resize_cache/iblock/d78/lj51v04z9x3y82f2ddh3kv5lgjrhau8c/200_200_1d7a58ff99b324185ccb5ad5dfbdb5e85/Arlekino.webp" itemprop="image"/>
<div class="itemcard__imgName" itemprop="name">Арлекино</div>
</div>
</div>
<div class="itemcard__name">Арлекино</div>
</a>


In [84]:
def get_character_data(info, idx, name):
  all_info = info.find_all('span', {'class':'characterPromo__propV'})

  rarity = len(all_info[0].find_all('i', {'class':'fa fa-star'}))

  eye = all_info[1].text.strip()
  weapon = all_info[2].text.strip()

  try:
    rating = info.find('div', {'title':'Общий рейтинг'}).text.strip()
  except:
    rating = None

  return {
      'index_character': idx,
      'name': name,
      'rarity': rarity,
      'gods_eye': eye,
      'weapon': weapon,
      'rating': rating
  }

In [85]:
items = []
characters = []
name = "Genshin Impact"
group = "Игры"
idx = 0
for item in itemcards:
  index_character = idx
  hero = item.get('title')

  character_url = genshin_url + list(filter(None, item.get('href').split('/')))[-1] + "/"
  character_html = requests.get(character_url).text
  info = BeautifulSoup(character_html, 'html')

  characters.append(get_character_data(info, index_character, hero))

  try:
    image_link = url_to_main_page + info.find('img', {'class':'characterPromo__image'}).get('src')
  except:
    image_link = "https://static.wikia.nocookie.net/genshin-impact/images/d/d8/Персонаж_Путешественник_Карточка.png/revision/latest/scale-to-width-down/1000?cb=20220907205632&path-prefix=ru"
  url = image_link

  items.append({'index_character': index_character,
              'name': name,
              'hero': hero,
              'url': url,
              'group': group})
  idx += 1
print(idx)

85


In [86]:
df = pd.DataFrame(items)
df

,index_character,name,hero,url,group
0,0,Genshin Impact,Арлекино,https://genshin-info.ru/upload/resize_cache/ib...,Игры
1,1,Genshin Impact,Тиори,https://genshin-info.ru/upload/resize_cache/ib...,Игры
2,2,Genshin Impact,Сянь Юнь,https://genshin-info.ru/upload/resize_cache/ib...,Игры
3,3,Genshin Impact,Ка Мин,https://genshin-info.ru/upload/resize_cache/ib...,Игры
4,4,Genshin Impact,Навия,https://genshin-info.ru/upload/resize_cache/ib...,Игры
...,...,...,...,...,...
80,80,Genshin Impact,Бэй Доу,https://genshin-info.ru/upload/resize_cache/ib...,Игры
81,81,Genshin Impact,Беннет,https://genshin-info.ru/upload/resize_cache/ib...,Игры
82,82,Genshin Impact,Барбара,https://genshin-info.ru/upload/resize_cache/ib...,Игры
83,83,Genshin Impact,Путешественник Гео,https://static.wikia.nocookie.net/genshin-impa...,Игры


In [45]:
df.to_excel('Genshin_Impact.xlsx', index = False)

In [87]:
df2 = pd.DataFrame(characters)
df2

,index_character,name,rarity,gods_eye,weapon,rating
0,0,Арлекино,5,Пиро,Копье,None
1,1,Тиори,5,Гео,Меч,A
2,2,Сянь Юнь,5,Анемо,Катализатор,S
3,3,Ка Мин,4,Пиро,Двуручный меч,S
4,4,Навия,5,Гео,Двуручный меч,S+
...,...,...,...,...,...,...
80,80,Бэй Доу,4,Электро,Двуручный меч,A
81,81,Беннет,4,Пиро,Меч,S+
82,82,Барбара,4,Гидро,Катализатор,C
83,83,Путешественник Гео,5,Гео,Меч,D


In [88]:
df2.to_excel('Genshin_Impact_Characters.xlsx', index = False)